In [9]:
!pip3 install plotly

     |████████████████████████████████| 13.2 MB 825 kB/s eta 0:00:01    |████████████████▌               | 6.8 MB 7.7 MB/s eta 0:00:01     |███████████████████             | 7.8 MB 7.7 MB/s eta 0:00:01     |███████████████████████████████ | 12.8 MB 7.7 MB/s eta 0:00:01


In [1]:
import os

import cv2
import thop # MACs, FLop 수 측정 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision import transforms

import optuna
from optuna.trial import TrialState

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_class = 10

LOG_INTERVAL = 600

In [3]:
def define_model(trial):
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []
    
    in_features = 28 * 28
    for i in range(n_layers):
        out_features = trial.suggest_int(f"n_units_l{i}", 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float(f"dropout_l{i}", 0.2, 0.5)
        layers.append(nn.Dropout(p))
        
        in_features = out_features

    layers.append(nn.Linear(in_features, num_class))
    layers.append(nn.LogSoftmax(dim=1))
    
    return nn.Sequential(*layers)

In [4]:
def get_dataloader(batch_size):
    train_dataset = datasets.FashionMNIST('./cifar10/', train=True, download=True, transform=transforms.ToTensor())
    valid_dataset = datasets.FashionMNIST('./cifar10/', train=False, transform=transforms.ToTensor())

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2
    )
    
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2
    )
    
    return train_loader, valid_loader

In [5]:
def train(model, optimizer, train_loader, epoch, batch_size):
    model.train()
    for batch_idx, (X_train, y_train) in enumerate(train_loader):
        X_train = X_train.view(X_train.size(0), -1).to(device)
        y_train = y_train.to(device)

        optimizer.zero_grad()
        output = model(X_train)
        loss = F.nll_loss(output, y_train)
        loss.backward()
        optimizer.step()

        if batch_idx % LOG_INTERVAL == 0:
            print(f"Train Epoch: {epoch} [{(batch_idx + 1) * batch_size}/{len(train_loader.dataset)}]\tLoss: {loss.item():.6f}")
            
def evaluation(model, valid_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for batch_idx, (X_valid, y_valid) in enumerate(valid_loader):
            X_valid = X_valid.view(X_valid.size(0), -1).to(device)
            y_valid = y_valid.to(device)

            output = model(X_valid)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(y_valid.view_as(pred)).sum().item()

    accuracy = correct / len(valid_loader.dataset)

    flops, _ = thop.profile(model, inputs=(torch.randn(1, 28 * 28).to(device),), verbose=False)

    return flops, accuracy

In [6]:
def objective(trial):
    model = define_model(trial).to(device)
    
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32, 64])
    
    train_loader, valid_loader = get_dataloader(batch_size)
    
    for epoch in range(5):
        train(model, optimizer, train_loader, epoch + 1, batch_size)
    flops, accuracy = evaluation(model, valid_loader)
    
    return flops, accuracy

In [10]:
study = optuna.create_study(directions=["minimize", "maximize"])
study.optimize(objective, n_trials=30, timeout=300)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

[I 2021-05-27 11:23:14,621] A new study created in memory with name: no-name-5dfdceca-7a48-4974-9033-2ff4b7197169


Train Epoch: 1 [8/60000]	Loss: 2.333590
Train Epoch: 1 [4808/60000]	Loss: 1.422965
Train Epoch: 1 [9608/60000]	Loss: 1.859233
Train Epoch: 1 [14408/60000]	Loss: 1.420964
Train Epoch: 1 [19208/60000]	Loss: 2.260689
Train Epoch: 1 [24008/60000]	Loss: 2.032409
Train Epoch: 1 [28808/60000]	Loss: 1.504974
Train Epoch: 1 [33608/60000]	Loss: 1.789898
Train Epoch: 1 [38408/60000]	Loss: 2.019917
Train Epoch: 1 [43208/60000]	Loss: 1.843495
Train Epoch: 1 [48008/60000]	Loss: 2.021448
Train Epoch: 1 [52808/60000]	Loss: 1.846212
Train Epoch: 1 [57608/60000]	Loss: 1.876760
Train Epoch: 2 [8/60000]	Loss: 1.617670
Train Epoch: 2 [4808/60000]	Loss: 2.374501
Train Epoch: 2 [9608/60000]	Loss: 1.942030
Train Epoch: 2 [14408/60000]	Loss: 1.708834
Train Epoch: 2 [19208/60000]	Loss: 1.605093
Train Epoch: 2 [24008/60000]	Loss: 1.818170
Train Epoch: 2 [28808/60000]	Loss: 1.787366
Train Epoch: 2 [33608/60000]	Loss: 1.991976
Train Epoch: 2 [38408/60000]	Loss: 1.641342
Train Epoch: 2 [43208/60000]	Loss: 1.772716


[I 2021-05-27 11:25:31,021] Trial 0 finished with values: [73860.0, 0.1974] and parameters: {'n_layers': 2, 'n_units_l0': 90, 'dropout_l0': 0.28474486906122964, 'n_units_l1': 33, 'dropout_l1': 0.3052251910646641, 'optimizer': 'Adam', 'lr': 0.02536785489117226, 'batch_size': 8}. 


Train Epoch: 1 [16/60000]	Loss: 2.298073
Train Epoch: 1 [9616/60000]	Loss: 2.315526
Train Epoch: 1 [19216/60000]	Loss: 2.313309
Train Epoch: 1 [28816/60000]	Loss: 2.290269
Train Epoch: 1 [38416/60000]	Loss: 2.285532
Train Epoch: 1 [48016/60000]	Loss: 2.315760
Train Epoch: 1 [57616/60000]	Loss: 2.357987
Train Epoch: 2 [16/60000]	Loss: 2.298936
Train Epoch: 2 [9616/60000]	Loss: 2.304021
Train Epoch: 2 [19216/60000]	Loss: 2.235815
Train Epoch: 2 [28816/60000]	Loss: 2.332672
Train Epoch: 2 [38416/60000]	Loss: 2.327435
Train Epoch: 2 [48016/60000]	Loss: 2.311598
Train Epoch: 2 [57616/60000]	Loss: 2.288959
Train Epoch: 3 [16/60000]	Loss: 2.300531
Train Epoch: 3 [9616/60000]	Loss: 2.313642
Train Epoch: 3 [19216/60000]	Loss: 2.364559
Train Epoch: 3 [28816/60000]	Loss: 2.330896
Train Epoch: 3 [38416/60000]	Loss: 2.282769
Train Epoch: 3 [48016/60000]	Loss: 2.273722
Train Epoch: 3 [57616/60000]	Loss: 2.305043
Train Epoch: 4 [16/60000]	Loss: 2.294547
Train Epoch: 4 [9616/60000]	Loss: 2.339313
Trai

[I 2021-05-27 11:26:33,265] Trial 1 finished with values: [16260.0, 0.1219] and parameters: {'n_layers': 3, 'n_units_l0': 6, 'dropout_l0': 0.2181598737427078, 'n_units_l1': 78, 'dropout_l1': 0.3278983383803069, 'n_units_l2': 126, 'dropout_l2': 0.25477744101607314, 'optimizer': 'SGD', 'lr': 3.6418976146596215e-05, 'batch_size': 16}. 


...

Train Epoch: 1 [32/60000]	Loss: 2.310396
Train Epoch: 1 [19232/60000]	Loss: 1.438598
Train Epoch: 1 [38432/60000]	Loss: 0.998718
Train Epoch: 1 [57632/60000]	Loss: 1.199026
Train Epoch: 2 [32/60000]	Loss: 0.714622
Train Epoch: 2 [19232/60000]	Loss: 1.268554
Train Epoch: 2 [38432/60000]	Loss: 1.069105
Train Epoch: 2 [57632/60000]	Loss: 1.062551
Train Epoch: 3 [32/60000]	Loss: 0.735945
Train Epoch: 3 [19232/60000]	Loss: 1.023360
Train Epoch: 3 [38432/60000]	Loss: 0.736868
Train Epoch: 3 [57632/60000]	Loss: 0.749549
Train Epoch: 4 [32/60000]	Loss: 0.973790
Train Epoch: 4 [19232/60000]	Loss: 0.960187
Train Epoch: 4 [38432/60000]	Loss: 0.894830
Train Epoch: 4 [57632/60000]	Loss: 0.804812
Train Epoch: 5 [32/60000]	Loss: 1.188487
Train Epoch: 5 [19232/60000]	Loss: 1.026403
Train Epoch: 5 [38432/60000]	Loss: 0.891758
Train Epoch: 5 [57632/60000]	Loss: 0.965979


[I 2021-05-27 11:28:03,206] Trial 3 finished with values: [30024.0, 0.746] and parameters: {'n_layers': 3, 'n_units_l0': 30, 'dropout_l0': 0.4362449644849893, 'n_units_l1': 74, 'dropout_l1': 0.26511731142938555, 'n_units_l2': 51, 'dropout_l2': 0.2865670114680688, 'optimizer': 'Adam', 'lr': 0.00660398163203898, 'batch_size': 32}. 


Train Epoch: 1 [8/60000]	Loss: 2.305089
Train Epoch: 1 [4808/60000]	Loss: 2.327289
Train Epoch: 1 [9608/60000]	Loss: 2.317467
Train Epoch: 1 [14408/60000]	Loss: 2.290693
Train Epoch: 1 [19208/60000]	Loss: 2.309203
Train Epoch: 1 [24008/60000]	Loss: 2.322999
Train Epoch: 1 [28808/60000]	Loss: 2.285873
Train Epoch: 1 [33608/60000]	Loss: 2.297134
Train Epoch: 1 [38408/60000]	Loss: 2.252794
Train Epoch: 1 [43208/60000]	Loss: 2.345644
Train Epoch: 1 [48008/60000]	Loss: 2.331042
Train Epoch: 1 [52808/60000]	Loss: 2.292424
Train Epoch: 1 [57608/60000]	Loss: 2.317260
Train Epoch: 2 [8/60000]	Loss: 2.317501
Train Epoch: 2 [4808/60000]	Loss: 2.337159
Train Epoch: 2 [9608/60000]	Loss: 2.299973
Train Epoch: 2 [14408/60000]	Loss: 2.276523
Train Epoch: 2 [19208/60000]	Loss: 2.268690
Train Epoch: 2 [24008/60000]	Loss: 2.307907
Train Epoch: 2 [28808/60000]	Loss: 2.321126
Train Epoch: 2 [33608/60000]	Loss: 2.287346
Train Epoch: 2 [38408/60000]	Loss: 2.299551
Train Epoch: 2 [43208/60000]	Loss: 2.332596


[I 2021-05-27 11:30:02,603] Trial 4 finished with values: [36948.0, 0.1134] and parameters: {'n_layers': 3, 'n_units_l0': 43, 'dropout_l0': 0.22017759078832386, 'n_units_l1': 29, 'dropout_l1': 0.4249698373164872, 'n_units_l2': 51, 'dropout_l2': 0.4442886436114746, 'optimizer': 'SGD', 'lr': 1.799256274814117e-05, 'batch_size': 8}. 


Study statistics: 
  Number of finished trials:  5
  Number of pruned trials:  0
  Number of complete trials:  5


In [13]:
optuna.visualization.plot_pareto_front(study, target_names=["FLOPS", "accuracy"])

<ipython-input-13-17920863a436>:1: ExperimentalWarning: plot_pareto_front is experimental (supported from v2.4.0). The interface can change in the future.
  optuna.visualization.plot_pareto_front(study, target_names=["FLOPS", "accuracy"])


ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.